In [30]:
import pickle
import numpy as np
import torch
from tqdm import tqdm

## Task I: Word-based CNN for Text Classification

### 1. Data

The dataset that we are going to use is the imdb dataset of movie reviews. These are labelled by sentiment (positive/negative). 

The reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers). 

For convenience, words are indexed by overall frequency in the dataset, so that for instance the integer "3" encodes the 3rd most frequent word in the data. This allows for quick filtering operations such as: "only consider the top 10,000 most common words, but eliminate the top 20 most common words".

More information regarding the dataset can be found in the official [documentation](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification).


In [8]:
# Load the data
train_samples, train_labels, test_samples, test_labels = pickle.load(open('./imdb.pkl', 'rb'))

print(len(train_samples), 'train sequences')
print(len(test_samples), 'test sequences')

25000 train sequences
25000 test sequences


### 2. Preprocess the text data 

In this particular case, where we are using the imdb dataset there is no need to do all the traditional preprocessings that we normally do when dealing with NLP problems. Part of them are already done at this point.

  - Split the dataset in train and test (maybe also validation).
  - Tokenize and transform to integer index. Here we would need to: 
    - instantitate a *Tokenizer()* object, 
    - fit that object on the text on which we are training the model (use the *fit_on_texts()* method)
    - call *texts_to_sequences()* for both the training and the test text.

  - **Add padding to ensure that all vectors have the same dimensionality.** Note that this is the only pre-processing that needs to be done in the case of the current imdb dataset.

In [9]:
desired_length = 1000

# TODO 2. Pad sequences

def pad(samples, length):
    return np.array([
        sample[:length] if len(sample) >= length else sample + [10001] * (length - len(sample))
        for sample in samples
    ])

train_samples = pad(train_samples, desired_length)
test_samples = pad(test_samples, desired_length)

print('train_samples shape:', train_samples.shape)
print('test_samples shape:', test_samples.shape)

train_samples shape: (25000, 1000)
test_samples shape: (25000, 1000)


### 3. CNN Architecture

A very simple example of CNN architecture for text classification can be found [here](https://keras.io/examples/imdb_cnn/). Take a look in order to have an idea about how things should look like at this level.

A few other simple implementations: https://www.kaggle.com/jacklinggu/keras-mlp-cnn-test-for-text-classification/notebook.

Please note that if the dataset is too small and the complexity of the model is large, then the phenomenon of overfitting will be imminent and fast.

The model will look similar to what you have seen in the CNN for image classification (the first lab):

> **3.1.** Instantiate the model. Use the constructor of the Sequential class as you have done in the first laboratory.

> **3.2.** After creating the initial model, we need to populate it with the hidden layers:
- **3.2.1.** With text classification, the first layer is usually an **embedding layer** in which word indices that come from the input layer are converted to word vectors (word2vec). This is an important conversion which enables a more efficient and faster processing of textual data. Each word integer is mapped into a one dimensional vector of floats which captures its syntactical properties within the movie reviews text corpus. This subject may be covered in a little more depth in an upcoming laboratory. At the moment, a few insights have been introduced in the theoretical support for this lab and even more explanation can be found in the Tensorflow and Google code archives and other blogs:
  - https://code.google.com/archive/p/word2vec
  - https://www.tensorflow.org/tutorials/word2vec
  - https://www.youtube.com/watch?v=T8tQZChniMk&list=PLR2RxXcwFe533vpJhgiDOAONyRzj3lzbJ
  - http://colah.github.io/posts/2014-07-NLP-RNNs-Representations
- **3.2.2.** After the embedding layer, add a **dropout** layer with a probability of 0.4.
- **3.2.3.** Add a **CONV** layer with 128 filters, filter size 3, padding same, activation relu and stride 1. Since our word vectors are one dimensional, we only need 1-dim convolutions. Keras provides us with a built in method for doing it elegantly. Note that we need to specify a convolution kernel length and number of filters to use (nb_filter). More info about parameters and usage at: https://keras.io/layers/convolutional/
- **3.2.4.** Use **Max Pooling** after the CONV layer.
- **3.2.5.** Add another **CONV** layer, similar in terms of parameters with what we have at 3.2.3. The only difference is that here we are trying a different kernel size (5 to be more specific).
- **3.2.6.** Add a **Batch Normalization** layer. You can, of course, try to train the model without this layer, or with any other modification. Without this layer in particular, you are going to (probably) see a fair amount of overfitting.
- **3.2.7.** Use **Max Pooling** again.
- **3.2.8.** Add a Flatten layer.
- **3.2.9.** Add a dense layer with hidden_dims hidden units and activation relu.
- **3.2.10. Dropout** neurons with a probability of 0.5. Do this in order to add even more prvention of overfitting.
- **3.2.11.** Our output layer consists of one neuron. The sigmoid activation will produce a float number between 0 and 1. We can round it to 0 or 1 to conclude if the movie review is negative or positive, or we can interpret it as a fuzzy value of how much the review is positive or negative.

> **3.3.** Compile the model using binary crossentropy as loss, accuracy as metric and adam optimizer. These values for the parameters should be seen as a suggestion and you are more than welcome to use any other valid values and see how the behavior changes.

> **3.4.** Train (fit) the model on the training data, with the test set as validation data. Set shuffle to True. For this particular dataset,  one possible (decent) batch size is 64 and (maybe) 10 epochs should be enough in order to dive into the problem.

In [42]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.emb = torch.nn.Embedding(10002, 100, padding_idx = 10001)
        
        self.net = torch.nn.Sequential(
            torch.nn.Dropout(.4),
            
            torch.nn.Conv1d(100, 128, kernel_size = 3, padding = 1),
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(2),
            
            torch.nn.Conv1d(128, 128, kernel_size = 5, padding = 2),
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(2),
            
            torch.nn.Conv1d(128, 128, kernel_size = 5, padding = 2),
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(2),
            
            torch.nn.AvgPool1d(125, stride = 125),
            
            torch.nn.Flatten(),
            
            torch.nn.Linear(128, 2)
        )
        
    def forward(self, input):
        output = self.emb(input).permute(0, 2, 1)
        output = self.net(output)
        return output
    
class Dataset(torch.utils.data.Dataset):
    def __init__(self, use = 'train'):
        self.samples = train_samples
        self.labels = train_labels
        if use == 'test':
            self.samples = test_samples
            self.labels = test_labels
            
    def __getitem__(self, sample_n):
        return self.samples[sample_n], self.labels[sample_n]
    
    def __len__(self):
        return len(self.samples)

model = Model().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
loss_f = torch.nn.CrossEntropyLoss()
train_dataset = Dataset('train')
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    num_workers = 20,
    shuffle = True,
    batch_size = 64
)
test_dataset = Dataset('test')
test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    num_workers = 20,
    shuffle = True,
    batch_size = 64
)

for epoch_n in range(10):
    model.train()
    for sample, label in tqdm(train_dataloader):
        model.zero_grad()

        sample = sample.long().cuda()
        label = label.cuda()

        output = model(sample)
        loss = loss_f(output, label)

        loss.backward()
        optimizer.step()

    model.eval()
    accs = []
    for sample, label in tqdm(test_dataloader):
        model.zero_grad()

        sample = sample.long().cuda()
        label = label.cuda()

        output = model(sample)
        accs.append((output.argmax(1) == label).float().mean().detach().cpu().numpy())
    print(np.mean(accs))


100%|██████████| 391/391 [00:01<00:00, 246.40it/s]


0.8418558


100%|██████████| 391/391 [00:01<00:00, 246.79it/s]


0.8248961


100%|██████████| 391/391 [00:01<00:00, 235.00it/s]


...

0.89909685


100%|██████████| 391/391 [00:01<00:00, 247.31it/s]


0.8971707


100%|██████████| 391/391 [00:01<00:00, 250.49it/s]

0.89537245
